## 第七章-PAC与梯度上升法

### 7-1 什么是PCA?

主成分分析法 `Principal Component Analysis`

- 一个非监督的机器学习算法
- 主要用于数据的降维
- 通过降维, 可以发现更便于人理解的特征
- 其他应用: 可视化, 降噪

![](./picture/7-1-1.png)

![](./picture/7-1-2.png)

<font face="Courier New">

目标: **找到一条使得样本间间距最大的轴**<br>
使用方差(`Variance`)定义样本间的间距, $Var(x) = \frac{1}{m}\sum^{m}_{i=1}(x_{i} - \overline{x})^{2}$<br>
方差:描述样本整体分布的疏密的指标, 方差越大, 样本间越稀疏, 方差越小样本间越紧密, 因此, 目标转化为**找到使得样本最大方差**

![](picture/7-1-3.png)

<font face="Courier New">

$Var(x) = \frac{1}{m}\sum^{m}_{i=1}(x_{i} - \overline{x})^{2}$;其中 $\overline{x}=0$; 则: $Var(x) = \frac{1}{m}\sum^{m}_{i=1}x_{i}^{2}$

<font face="Courier New">

对所有的样本进行 `demean` 处理, 我们想要求一个轴的方向 `w = (w1, w2)`, 使得我们所有的样本, 映射到 w 以后, 得到: $$Var(X_{project}) = \frac{1}{m}\sum^{m}_{i=1}(X_{project}^{(i)} - \overline{X}_{project})^{2} ;其中X_{project}为映射后的样本.$$
$$Var(X_{project}) = \frac{1}{m}\sum^{m}_{i=1}||X_{project}^{(i)} - \overline{X}_{project}||^{2}$$
$$Var(X_{project}) = \frac{1}{m}\sum^{m}_{i=1}||X_{project}^{(i)}||^{2}$$
使得上述最后一个式子结果最大

![](picture/7-1-4.png)

![](picture/7-1-5.png)

**主成分分析与线性回归区别**

![](picture/7-1-6.png)

![](picture/7-1-7.png)

### 7-2 使用梯度上升法求解PCA问题

![](picture/7-1-6.png)